In [1]:
import os
import tensorflow as tf
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
import emoji
from soynlp.normalizer import repeat_normalize
import pandas as pd
from tensorflow.keras.layers import Input, Dense, concatenate, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.layers import Activation
from sklearn.model_selection import train_test_split

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from utils.pickles import *
import pandas as pd
import numpy as np
from utils.data_frame import DataFrame
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
emb_model = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')
from utils.clean import review_clean
import wandb


In [3]:
model = tf.keras.models.load_model('models/mlp_best.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 469)               360661    
                                                                 
 batch_normalization (BatchN  (None, 469)              1876      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 469)               0         
                                                                 
 dropout (Dropout)           (None, 469)               0         
                                                                 
 dense_1 (Dense)             (None, 241)               113270    
                                                                 
 batch_normalization_1 (Batc  (None, 241)              964       
 hNormalization)                                        

2021-11-18 16:56:57.926617: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-18 16:56:58.548814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7757 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:09:00.0, compute capability: 8.6
2021-11-18 16:56:58.549640: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10094 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [4]:
kurly_emb = read_pickle_files('kurly_review_emb_mlp.pickle')
emart_emb = read_pickle_files('emart_review_emb_mlp.pickle')

In [12]:
kurly_pred = model.predict(kurly_emb)
emart_pred = model.predict(emart_emb)

In [13]:
kurly_pred

array([[0.9351257 ],
       [0.6669641 ],
       [0.982017  ],
       ...,
       [0.98068947],
       [0.8122823 ],
       [0.8679948 ]], dtype=float32)

In [14]:
kurly_pred = list(map(lambda x: x[0], kurly_pred))
emart_pred = list(map(lambda x: x[0], emart_pred))

[0.9351257,
 0.6669641,
 0.982017,
 0.9843919,
 0.08259776,
 0.88365096,
 0.34952912,
 0.96428686,
 0.37375507,
 0.31184757,
 0.9991246,
 0.614636,
 0.96328926,
 0.5362188,
 0.9696951,
 0.83947384,
 0.07022804,
 0.715412,
 0.99910754,
 0.93470854,
 0.650519,
 0.6181845,
 0.6585859,
 0.6640835,
 0.74174917,
 0.7499833,
 0.8317772,
 0.9138418,
 0.82874745,
 0.022687044,
 0.9721683,
 0.862456,
 0.99586284,
 0.30892268,
 0.529569,
 0.9627851,
 0.8873059,
 0.8967122,
 0.8683355,
 0.8694534,
 0.7679212,
 0.707696,
 0.22954023,
 0.9915005,
 0.777211,
 0.8503656,
 0.61286676,
 0.98553586,
 0.9202384,
 0.9493395,
 0.8923514,
 0.99219465,
 0.9698145,
 0.96817464,
 0.7828535,
 0.73949564,
 0.8898214,
 0.9500782,
 0.11007625,
 0.6580664,
 0.99210584,
 0.9564653,
 0.4213086,
 0.9847348,
 0.08478902,
 0.98991674,
 0.8490109,
 0.6941296,
 0.80988425,
 0.95112437,
 0.92363113,
 0.60550886,
 0.7370168,
 0.7144167,
 0.30822197,
 0.9564987,
 0.72937757,
 0.94125843,
 0.9926611,
 0.6361669,
 0.88985574,
 

In [15]:
kurly_pred = pd.Series(kurly_pred)
emart_pred = pd.Series(emart_pred)
kurly_pred[:3]

0    0.935126
1    0.666964
2    0.982017
dtype: float32

In [ ]:
write_pickle_files(kurly_pred, 'kurly_pred_mlp.pickle')
write_pickle_files(emart_pred, 'emart_pred_mlp.pickle')

kurly_review = DataFrame('마켓컬리', '리뷰').get_df()
kurly_review.head()

In [4]:
kurly_review['refined_contest'] = kurly_review['content'].apply(review_clean)
kurly_review.head()

,name,user,star,content,refined_contest
0,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,황*현,,지난번 괜찮었던 기억으로 재구매 했는데 다른 브랜드였나 싶게 기름이 많어도 늠~ 많아요,지난번 괜찮었던 기억으로 재구매 했는데 다른 브랜드였나 싶게 기름이 많어도 늠~ 많아요
1,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,조*정,,이전에 먹었는데 맛있어서 재구매합니다. 전에는 고기가 두꺼운데 팬에 굽느라 냄새도 ...,이전에 먹었는데 맛있어서 재구매합니다. 전에는 고기가 두꺼운데 팬에 굽느라 냄새도 ...
2,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,유*,,정망 맛있게 먹었어요 부모님도 좋아하시네요~,정망 맛있게 먹었어요 부모님도 좋아하시네요~
3,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,남*인,,더니 너무 잘 먹어 또 쥬문요,더니 너무 잘 먹어 또 쥬문요
4,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,이*주,,맛은 괜찮아요 근데 지방이 좀 많네요. 에어프라이어 돌려 드세요.,맛은 괜찮아요 근데 지방이 좀 많네요. 에어프라이어 돌려 드세요.


In [7]:
kurly_review_emb = emb_model.encode(kurly_review['refined_contest'].values)
kurly_review_emb[:5]

array([[-0.36452207, -0.32493833, -1.1862199 , ...,  0.35337588,
        -0.14369754,  0.24113384],
       [ 0.04701273, -0.08883723, -0.7207835 , ...,  0.05653987,
        -0.35312185,  0.1911716 ],
       [-1.0835663 , -0.8103261 ,  0.22853942, ..., -0.6862127 ,
        -0.28821766, -0.208606  ],
       [-1.4373263 , -0.5712458 , -1.1832143 , ..., -0.23956299,
        -0.09101021, -0.37323034],
       [ 0.43890837, -0.15302145, -0.25606093, ...,  0.04977729,
         0.42403668,  0.38685054]], dtype=float32)

In [8]:
write_pickle_files(kurly_review_emb, 'kurly_review_emb_mlp.pickle')

In [9]:
emart_review = DataFrame('이마트몰', '리뷰').get_df()
emart_review.head()

,name,user,star,content
0,모짜렐라 비프라자냐,moo*******,0,흠.. 나는 질보다 양이다 vs ●나는 양보다 질이다● 둘중 어느부류신가요? 이질문...
1,모짜렐라 비프라자냐,jks*******,1,조리법이 아주 간단해서 만족합니다.토마토스파게티 소스에 소고기를 볶은 것 같은데 아...
2,모짜렐라 비프라자냐,des*******,0,"상품이 떨어졌었는지, 플라스틱이 깨져왔지만, 분명 새상품 임에도 박스 밑쪽에 음식찌..."
3,모짜렐라 비프라자냐,pia*******,0,전자렌지에 좀 녹인 후 위에 비닐 떼고 오븐에 돌렸어요. 맛이 괜찮긴 한데 기대했던...
4,모짜렐라 비프라자냐,tae*******,1,파스타는 쉽게 해먹는데 라자냐는 쉽게 해먹기는 어렵잖아요. 할인 하길래 한번 사봤는...


In [10]:
emart_review['refined_contest'] = emart_review['content'].apply(review_clean)
emart_review.head()

,name,user,star,content,refined_contest
0,모짜렐라 비프라자냐,moo*******,0,흠.. 나는 질보다 양이다 vs ●나는 양보다 질이다● 둘중 어느부류신가요? 이질문...,흠.. 나는 질보다 양이다 vs 나는 양보다 질이다 둘중 어느부류신가요? 이질문만봐...
1,모짜렐라 비프라자냐,jks*******,1,조리법이 아주 간단해서 만족합니다.토마토스파게티 소스에 소고기를 볶은 것 같은데 아...,조리법이 아주 간단해서 만족합니다.토마토스파게티 소스에 소고기를 볶은 것 같은데 아...
2,모짜렐라 비프라자냐,des*******,0,"상품이 떨어졌었는지, 플라스틱이 깨져왔지만, 분명 새상품 임에도 박스 밑쪽에 음식찌...","상품이 떨어졌었는지, 플라스틱이 깨져왔지만, 분명 새상품 임에도 박스 밑쪽에 음식찌..."
3,모짜렐라 비프라자냐,pia*******,0,전자렌지에 좀 녹인 후 위에 비닐 떼고 오븐에 돌렸어요. 맛이 괜찮긴 한데 기대했던...,전자렌지에 좀 녹인 후 위에 비닐 떼고 오븐에 돌렸어요. 맛이 괜찮긴 한데 기대했던...
4,모짜렐라 비프라자냐,tae*******,1,파스타는 쉽게 해먹는데 라자냐는 쉽게 해먹기는 어렵잖아요. 할인 하길래 한번 사봤는...,파스타는 쉽게 해먹는데 라자냐는 쉽게 해먹기는 어렵잖아요. 할인 하길래 한번 사봤는...


In [11]:
emart_review_emb = emb_model.encode(emart_review['refined_contest'].values)
write_pickle_files(emart_review_emb, 'emart_review_emb_mlp.pickle')